In [1]:
import pandas as pd
from m2p import PolyMaker
from polyid import generate_hash
from fingerprints import HierarchticalFingerprints

pm = PolyMaker()
hfp = HierarchticalFingerprints()

%load_ext autoreload
%autoreload 2

2023-03-31 10:14:16.149791: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 10:14:16.505736: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64::/home/wilsoa6/miniconda3/envs/stonks/lib/
2023-03-31 10:14:16.505935: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64::/home/wilsoa6/miniconda

# Generate Structures

In [2]:
# import data
data = pd.read_csv('../data/example_monomer_smiles.csv',index_col=0)

# generate monomers tuple
data['monomers'] = data.smiles_monomer.apply(pm.get_monomers)

# # generate polymer structures with DP=8 and 2 replicate stuctures
data = pm.thermoplastic(data.sample(5),DP=8,mechanism='all',replicate_structures=1)

# remove any polymers which may have resulted in an error during reaction
dferror = data[data.smiles_polymer.str.contains('ERR')]
data = data[~data.smiles_polymer.str.contains('ERR')]

# hash columns for tracking predictions and structures
data = generate_hash(data,hash_cols=['monomers','replicate_structure'])
data = generate_hash(data.reset_index(),hash_cols=['monomers'])

data.head()

100%|██████████| 5/5 [00:00<00:00, 35.74it/s]


,hash-monomers-replicate_structure,smiles_monomer,monomers,distribution,replicate_structure,smiles_polymer,mechanism
hash-monomers,,,,,,,
8bzgrkrCMEwiRmDaAXUuxk,3nsQX984voZRb9ZWKVjy35,CC(O)CCO.O=C(O)C(=O)O,"('O=C(O)C(=O)O', 'CC(O)CCO')",[],0,CC(CCOC(=O)C(=O)O)OC(=O)C(=O)OCCC(C)OC(=O)C(=O...,ester
hFFFZMShuePLGi9QDp8wgn,7PKBrjqjQqyxhZhu34MgFb,CC1CC(CC2CCC(N)C(C)C2)CCC1N.NCCCCCCCCCCCC(=O)O...,"('O=C(O)c1cccc(C(=O)O)c1', 'NCCCCCCCCCCCC(=O)O...",[],0,CC1CC(CC2CCC(NC(=O)c3cccc(C(=O)NC4CCC(CC5CCC(N...,amide
3qeQR2XjhmQtMkCuudzaNt,3MBddDcJuREXB2mUvqj49t,O=C(O)c1ccc(C(=O)O)o1.OCCO,"('OCCO', 'O=C(O)c1ccc(C(=O)O)o1')",[],0,O=C(O)c1ccc(C(=O)OCCOC(=O)c2ccc(C(=O)OCCOC(=O)...,ester
AGwyJqJLZBD9mAfeSfctEU,WMEMN8MMnAuD6uaALBXhTA,CC(O)CO.O=C(O)CCCCCCCC(=O)O,"('CC(O)CO', 'O=C(O)CCCCCCCC(=O)O')",[],0,CC(CO)OC(=O)CCCCCCCC(=O)OCC(C)OC(=O)CCCCCCCC(=...,ester
XKpj9NHHAD722ocTC2kVLf,LDUCu4LRs2Ny6kbdcnkH77,O=C(O)CCSCCC(=O)O.OCCO,"('OCCO', 'O=C(O)CCSCCC(=O)O')",[],0,O=C(O)CCSCCC(=O)OCCOC(=O)CCSCCC(=O)OCCOC(=O)CC...,ester


# Generate Atomic and Molecular/Morphological Fingerprints

In [3]:
hfp.gen_fp_atomic_molecular_morphological(data)

print('Atomic FPs\n{}\n'.format(hfp.df_atomic.to_markdown()))
print('Molecular and Morpholotical (mordred) FPs\n{}\n'.format(hfp.df_molecular_morphological.to_markdown()))
print('Atomic, Molecular/Morpholotical (mordred) FPs\n{}\n'.format(hfp.df_atomic_molecular_morphological.to_markdown()))

100%|██████████| 5/5 [00:03<00:00,  1.65it/s]

Atomic FPs
| hash-smiles_polymer    |   98513984 |   126685494 |   128522177 |   161963127 |   203616823 |   205323193 |   227001719 |   231600494 |   311968854 |   517457164 |   649201019 |   711859603 |   717512901 |   779300257 |   787069595 |   787341104 |   847957139 |   847961216 |   851463431 |   864662311 |   864674487 |   864942730 |   951226070 |   994485099 |   998646794 |   998869300 |   1026928756 |   1054767590 |   1167322652 |   1189967413 |   1236995907 |   1390349538 |   1403876697 |   1408936613 |   1489948064 |   1491315937 |   1510328189 |   1510461303 |   1533864325 |   1535166686 |   1583799011 |   1671178153 |   1682265313 |   1686095578 |   1693331843 |   1710869618 |   1730555933 |   1817700328 |   1858577693 |   1931471364 |   1998475861 |   2048616743 |   2053558574 |   2071782123 |   2081238858 |   2113816892 |   2117068077 |   2150151661 |   2157595070 |   2222715027 |   2229740503 |   2239854846 |   2245273601 |   2245384272 |   2246699815 |   2246728737 |